In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [47]:
# api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars=200)
# wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars=200)


def wiki_tool_fn(query: str) -> str:
    return api_wrapper.run(query)


wiki_tool = Tool(
    name="wiki",
    func=wiki_tool_fn,
    description="Use this to get Wikipedia information. Input is a search query string.",
)

In [48]:
wiki_tool.name

'wiki'

In [49]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
# from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
).split_documents(docs)

vectorDB = FAISS.from_documents(documents, OllamaEmbeddings(model="mxbai-embed-large"))
retriever = vectorDB.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x771251854110>, search_kwargs={})

In [53]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for onformation about LangSmith. For any questions about LangSmith, you must use this tool."
)

In [54]:
retrieval_tool.name

'langsmith_search'

In [55]:
# ## Arxiv Tool
# from langchain_community.utilities import ArxivAPIWrapper
# from langchain_community.tools import ArxivQueryRun

# arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars=200)
# arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
# arxiv.name

from langchain_community.utilities import ArxivAPIWrapper
from langchain.tools import Tool

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars=200)

def arxiv_tool_fn(query: str) -> str:
    return arxiv_wrapper.run(query)

arxiv_tool = Tool(
    name="arxiv",
    func=arxiv_tool_fn,
    description="Search Arxiv for research papers. Input should be a search query string.",
)

In [74]:
tools = [wiki_tool, arxiv_tool, retrieval_tool]
tools

[Tool(name='wiki', description='Use this to get Wikipedia information. Input is a search query string.', func=<function wiki_tool_fn at 0x771289e6b100>),
 Tool(name='arxiv', description='Search Arxiv for research papers. Input should be a search query string.', func=<function arxiv_tool_fn at 0x771289e6ae80>),
 Tool(name='langsmith_search', description='Search for onformation about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7712880ad4e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x771251854110>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _a

In [75]:
# #### only works with OpenAI
# from langchain import hub
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

In [80]:
from langchain.agents import initialize_agent, AgentType

llm = ChatOllama(model="gemma3:1b")

# Initialize the agent with correct agent type
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Run the agent
agent.run("Tell me about LangSmith")



> Entering new AgentExecutor chain...
```json
{
  "action": "Final Answer",
  "action_input": "LangSmith is a powerful search engine for scientific literature. It’s designed to help researchers quickly find relevant papers across various disciplines. Here’s a breakdown of what it offers:\n\n* **Focus on Scientific Papers:** It specializes in scientific literature, particularly papers from ArXiv, PubMed, and Scopus.  It aims to prioritize papers related to physics, chemistry, biology, and engineering.\n* **Advanced Search Capabilities:**  LangSmith employs a sophisticated index to understand the connections between papers. It allows you to search for concepts, authors, keywords, and even relationships between papers – going beyond simple keyword matching.\n* **Multiple Search Modes:**  It supports various search modes including keyword search, concept search, and query-based search.  You can even define ‘queries’ to narrow down results to a specific focus.\n* **Cross-Document Linking:

"LangSmith is a powerful search engine for scientific literature. It’s designed to help researchers quickly find relevant papers across various disciplines. Here’s a breakdown of what it offers:\n\n* **Focus on Scientific Papers:** It specializes in scientific literature, particularly papers from ArXiv, PubMed, and Scopus.  It aims to prioritize papers related to physics, chemistry, biology, and engineering.\n* **Advanced Search Capabilities:**  LangSmith employs a sophisticated index to understand the connections between papers. It allows you to search for concepts, authors, keywords, and even relationships between papers – going beyond simple keyword matching.\n* **Multiple Search Modes:**  It supports various search modes including keyword search, concept search, and query-based search.  You can even define ‘queries’ to narrow down results to a specific focus.\n* **Cross-Document Linking:** LangSmith is designed to provide cross-linking between papers – helping you understand connec

In [86]:
# agent.run("Tell me about Machine Learning")
agent.invoke("Tell me about Machine Learning")



> Entering new AgentExecutor chain...
```json
{
  "action": "Final Answer",
  "action_input": "Machine Learning is a field of artificial intelligence that focuses on enabling computer systems to learn from data without being explicitly programmed. Instead of relying on pre-defined rules, machine learning algorithms analyze data to identify patterns, make predictions, and improve their performance over time. Here’s a breakdown of key aspects:\n\n* **Core Idea:** Machine learning algorithms learn from data, much like humans do, by identifying correlations and trends.\n* **Types of Machine Learning:** There are several types, including:\n    * **Supervised Learning:** The algorithm learns from labeled data (input and desired output). Examples: classification (predicting categories) and regression (predicting a continuous value).\n    * **Unsupervised Learning:** The algorithm learns from unlabeled data and discovers hidden patterns.\n    * **Reinforcement Learning:** The algorithm learn

{'input': 'Tell me about Machine Learning',
 'output': 'Machine Learning is a field of artificial intelligence that focuses on enabling computer systems to learn from data without being explicitly programmed. Instead of relying on pre-defined rules, machine learning algorithms analyze data to identify patterns, make predictions, and improve their performance over time. Here’s a breakdown of key aspects:\n\n* **Core Idea:** Machine learning algorithms learn from data, much like humans do, by identifying correlations and trends.\n* **Types of Machine Learning:** There are several types, including:\n    * **Supervised Learning:** The algorithm learns from labeled data (input and desired output). Examples: classification (predicting categories) and regression (predicting a continuous value).\n    * **Unsupervised Learning:** The algorithm learns from unlabeled data and discovers hidden patterns.\n    * **Reinforcement Learning:** The algorithm learns by trial and error, receiving rewards o

In [87]:
agent.invoke("From Arxiv, give me a roadmap to learn reinforcement learning, keep in mind that i'm already good at machine learning and deep learning, but I don't know much about RL.")



> Entering new AgentExecutor chain...
```json
{
  "action": "arxiv",
  "action_input": "Reinforcement Learning Roadmap for Machine Learning & Deep Learning Enthusiasts",
  "thought": "Given your existing skills, a focused roadmap is best. Here’s a breakdown, emphasizing understanding the core concepts of RL and how they relate to your background:  It's structured around building foundational knowledge, then focusing on key algorithms and frameworks.  We’ll prioritize practical application.",
  "action": "langsmith_search",
  "query": "Reinforcement Learning Roadmap for Machine Learning & Deep Learning Enthusiasts",
  "thought": "I'll use this to locate resources suitable for this audience.  It will be a preliminary guide.  Let's start with a broad overview and then delve deeper into specific areas.",
  "action": "Final Answer",
  "action": "Final Answer",
  "thought": "I'll provide a basic roadmap with resource suggestions. Let’s start with a general outline.",
  "action": "wiki",
  

{'input': "From Arxiv, give me a roadmap to learn reinforcement learning, keep in mind that i'm already good at machine learning and deep learning, but I don't know much about RL.",
 'output': "Okay, let’s start with a foundational roadmap. Given your existing ML/DL background, we'll focus on building a solid understanding of core concepts before diving into algorithms.\n\n**Phase 1: Foundations – Understanding the Basics**\n\n1. **Markov Decision Processes (MDPs):**  A deep understanding of MDPs is crucial. Learn the components – states, actions, rewards, transition probabilities, and discount factors.  This will provide a conceptual framework for how RL works. Resources: [arxiv](https://arxiv.org/abs/?)\n2. **Q-Learning:**  This is the cornerstone of many RL algorithms.  Understand the Q-table, Q-values, exploration vs. exploitation, and the Bellman equation. [wiki](https://en.wikipedia.org/wiki/Q-learning)\n3. **Deep Q-Networks (DQN):**  DQN is a practical application of Q-learning.